<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">


# **Projeto final: Python para análise de dados**
## Analise de comportamente de clientes para aumento do limite de credito

---


>Este projeto tem por objetivo detectar padroes de comportamento que permitam o aumento do limite de creditos dos clientes adimplentes


---

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv', na_values='na')

In [ ]:
df.head(10)

---
#**Estrutura**

In [ ]:
df.shape #quantidade de linha e quantidade de colunas

In [ ]:
df[df['default'] == 0].shape #quantidade de linha e quantidade de colunas de clientes adimplentes

In [ ]:
df[df['default'] == 1].shape  #quantidade de linha e quantidade de colunas de clientes inadimplentes

In [ ]:
qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default'] == 0].shape
qtd_inadimplentes, _ = df[df['default'] == 1].shape

In [ ]:
print(f"A proporcão clientes adimplentes é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A proporcão clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")

#**Schema**

In [ ]:
df.head(5)

- Colunas e tipos de dados

In [ ]:
df.dtypes

- Atributos categoricos

In [ ]:
df.select_dtypes('object').describe().transpose()

- atributos numericos

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

# **Dados faltantes**

In [ ]:
df.head()

- Verificando quais colunas possuem dados faltantes

In [ ]:
df.isna().any()

---

- Estatisticas sobre os dados faltantes

In [ ]:
def dados_faltantes(df: pd.DataFrame) -> None:
   
   dados_faltantes = []
   for col in df.columns:
     if df[col].isna().any():
      qtd, _ = df[df[col].isna()].shape
      total, _ = df.shape
      dict_dados_faltantes = {col: {'quantidade': qtd, "porcentagem": round(100 * qtd/total, 2)}}
      dados_faltantes.append(dict_dados_faltantes)

   for stat in dados_faltantes:
    print(stat)

In [ ]:
dados_faltantes(df=df)

In [ ]:
dados_faltantes(df=df[df['default'] == 0]) #adimplentes

In [ ]:
dados_faltantes(df=df[df['default'] == 1]) #inadimplentes

# **Limpeza do dados e correção do schema**

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].dtypes

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].head(5)

funçao lambda para limpar os dados

In [ ]:
fn = lambda valor: float(valor.replace(".", "").replace(",", "."))

In [ ]:
df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(fn)
df['limite_credito'] = df['limite_credito'].apply(fn)

In [ ]:
df.dtypes

# **Remoção dos dados faltantes**

In [ ]:
df.dropna(inplace=True)

analisando a estrutura novamente

In [ ]:
df.shape

In [ ]:
df[df['default'] == 0 ].shape #adimplentes

In [ ]:
df[df['default'] == 1 ].shape #inadiplentes

In [ ]:
qtd_total_novo, _ = df.shape
qtd_adimplentes_novo, _ = df[df['default'] == 0].shape
qtd_inadimplentes_novo, _ = df[df['default'] == 1].shape

print(f"A proporcão adimplentes ativos é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A nova proporcão de clientes adimplentes é de {round(100 * qtd_adimplentes_novo / qtd_total_novo, 2)}%")
print("")
print(f"A proporcão clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")
print(f"A nova proporcão de clientes inadimplentes é de {round(100 * qtd_inadimplentes_novo / qtd_total_novo, 2)}%")

# **Visualização dos dados**

In [ ]:
sns.set_style("darkgrid")

separando os adinplentes e inadimplentes

In [ ]:
df_adimplentes = df[df['default'] == 0] #adimplentes

In [ ]:
df_inadimplentes = df[df['default'] == 1] #inadimplentes

- comparando variaveis categoricas

salario anual

In [ ]:
coluna = 'salario_anual'
titulos = ['salario anual dos Clientes', 'salario anual dos Clientes Adimplentes', 'salario anual dos Clientes Inadimplentes']

eixo = 0
max_y = 0
max = df.select_dtypes('object').describe()[coluna]['freq'] * 1.1

figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplentes, df_inadimplentes]:

  df_to_plot = dataframe[coluna].value_counts().to_frame()
  df_to_plot.rename(columns={coluna: 'frequencia_absoluta'}, inplace=True)
  df_to_plot[coluna] = df_to_plot.index
  df_to_plot.sort_values(by=[coluna], inplace=True)
  df_to_plot.sort_values(by=[coluna])

  f = sns.barplot(x=df_to_plot[coluna], y=df_to_plot['frequencia_absoluta'], ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
  f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

- Comparando variaveis numericas

limite de crédito

In [ ]:
coluna = 'limite_credito'
titulos = ['Limite de credito', 'Limite de credito de Adimplentes', 'Limite de credito de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplentes, df_inadimplentes]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

Transações nos ultimos 12 meses

In [ ]:
coluna = 'qtd_transacoes_12m'
titulos = ['Transações no Último Ano', 'Transações no Último Ano de Adimplentes', 'Transações no Último Ano de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplentes, df_inadimplentes]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

- comparação valores

In [ ]:
f = sns.relplot(x='valor_transacoes_12m', y='limite_credito', data=df, hue='default')
_ = f.set(title='Relação entre valor das transações e limites de credito no Último Ano', )

In [ ]:
f = sns.relplot(x='qtd_transacoes_12m', y='limite_credito', data=df, hue='default')
_ = f.set(title='Relação entre quantidade de transações e limites de credito no Último Ano', )

# **Resumo**

>Apesar de vários intervalos diferentes podemos notar com essa analise visual que clientes que fazem transaçoes com valor entre 6 e 10 mil por exemplo é seguro aumentar o limite até aproximadamente 4 mil.
Para valores entre 3 e 4 mil o aumento pode ser maior sem risco de inadimplencia.
Clientes com 90 ou mais transações ao ano o risco de inadimplencia é praticamente nulo, clientes com limite superior a 20 mil e com 90 transaçoes ou menos ao ano o risco de inadimplencia é moderado, limite superior a 34 mil aumenta para alto risco de inadimplencia